## GENERA LOTES DE DATOS PARA EVALUACIÖN DE ALGORITMOS.

Generar Archivos de estimaciones para comparar efectivida

# IMPORTACIONES BÁSICAS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyodbc
from dotenv import dotenv_values

# Mostrar el DataFrame resultante
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

# LIBRERIAS NECESARIAS 
import base64
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Tiempo y Performance
from datetime import datetime, timedelta
import time  # Para medir el tiempo de ejecución
import getpass  # Para obtener el usuario del sistema operativo
import uuid  # Importar la librería uuid

# Mostrar el DataFrame resultante
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

# Parámetros de conexión a la base de datos
secrets = dotenv_values(".env")   # Connection String from .env
folder = secrets["FOLDER_DATOS"]


# Solo importa lo necesario desde el módulo de funciones
from funciones_forecast import (
    get_forecast,
    Open_Connection,
    Close_Connection,
    Open_Diarco_Data,
    generar_datos,
    Procesar_ALGO_01,
    Procesar_ALGO_02,
    Procesar_ALGO_03,
    Procesar_ALGO_04,
    Procesar_ALGO_05,
    Procesar_ALGO_06,    
    generar_datos,    
    get_execution_execute_by_status,
    get_full_parameters,
    update_execution,
    update_execution_execute
)

# FUNCIONES LOCALES

def dividir_dataframe(data, fecha_corte):
    """
    Divide un DataFrame en dos partes: data_train y data_test según la fecha_corte.
    
    :param data: DataFrame con la columna 'Fecha'
    :param fecha_corte: Fecha límite para dividir el DataFrame (tipo datetime o string con formato 'YYYY-MM-DD')
    :return: data_train, data_test
    """
    # Asegurarse de que la columna 'Fecha' sea de tipo datetime
    data['Fecha'] = pd.to_datetime(data['Fecha'])
    
    # Filtrar los datos
    data_train = data[data['Fecha'] < pd.to_datetime(fecha_corte)]
    data_test = data[data['Fecha'] >= pd.to_datetime(fecha_corte)]
    
    return data_train, data_test


# CARGAR DATOS DE LA DEMANDA

### 1) Cargar Maestro de Sucursales


In [20]:
# CARGAR DESDE BASE DE DATOS
conn = Open_Connection()
label = 'T100_Sucursales'

# Consulta SQL para obtener las ventas de un proveedor específico   
# Reemplazar {proveedor} en la consulta con el ID de la tienda actual
query = f"""
SELECT [C_SUCU_EMPR] as id_tienda
      ,[N_SUCURSAL] as n_sucursal
      ,[N_SUCURSAL_ABREV] as n_sucursal_abrev
      ,[N_SUCURSAL_ABREV2] as n_sucursal_abrev2
      ,[N_SUCURSAL_ABREV3]as n_sucursal_abrev3    
      ,[N_CALLE] as n_calle
      ,[N_LOCALIDAD] as n_localidad
      ,[C_POSTAL_INM] as c_postal
      ,[C_PROVINCIA_SUCU] as c_provincia
      ,[C_ZONA] as c_zona
      ,[C_ZONA_REGIONAL] as c_region
      ,[CLUSTER_LOG] as cluster_log
      ,[C_FORMATO] as c_formato
      ,[FORMATO] as formato
      ,[M_BAJA] as m_baja
      ,[ETIQUETA] AS etiqueta
  FROM [data-sync].[dbo].[T_100_SUCURSALES];
"""

# Ejecutar la consulta SQL
data = pd.read_sql(query, conn)

# Adecuar Tipos de Datos
data['c_provincia']= data['c_provincia'].astype(int)
data['c_zona']= data['c_zona'].astype(int)
data['c_region']= data['c_region'].astype(int)

# Guardar los resultados en un archivo CSV con el nombre de la tienda
file_path = f'data/{label}.csv'
data.to_csv(file_path, index=False, encoding='utf-8')

print(f"Archivo guardado: {file_path}")

# Cerrar la conexión después de la iteración
conn.close()

Archivo guardado: data/T100_Sucursales.csv


In [ ]:
# CARGAR DESDE CSV
import chardet

with open('data/T100_Sucursales.csv', 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])  # Muestra la codificación detectada

# Cargar el archivo con codificación UTF-8
sucursales = pd.read_csv('data/T100_Sucursales.csv', encoding='utf-8', sep=',')

# Visualización de las primeras filas\
sucursales.head()

### 2) Cargar Maestro de Proveedores

In [ ]:
# CARGAR DESDE BASE DE DATOS
conn = Open_Connection()
label = 'T020_Proveedores'

# Consulta SQL para obtener las ventas de un proveedor específico   
# Reemplazar {proveedor} en la consulta con el ID de la tienda actual
query = f"""
SELECT [C_PROVEEDOR]
      ,[N_PROVEEDOR]
      ,[C_TIPO_PROVEEDOR]
      ,[C_TIPO_PROVEEDOR_DIARCO]
      ,[F_ALTA]
      ,[F_BAJA]
      ,[M_ACTIVO]
      ,[C_LINEA_PRODUCTO]
      ,[C_EANEDI]
      ,[C_PLAZO_ENTREGA1]
      ,[C_PLAZO_ENTREGA2]
      ,[C_PLAZO_ENTREGA3]
      ,[C_PLAZO_ENTREGA4]
      ,[C_PLAZO_ENTREGA5]
      ,[C_PLAZO_ENTREGA6]
      ,[M_ATEN_CLIE_LUN]
      ,[M_ATEN_CLIE_MAR]
      ,[M_ATEN_CLIE_MIE]
      ,[M_ATEN_CLIE_JUE]
      ,[M_ATEN_CLIE_VIE]
      ,[M_ATEN_CLIE_SAB]
      ,[M_ATEN_CLIE_DOM]
      ,[I_ENVASE]
      ,[M_BAJA]
      ,[U_DIAS_TOPE_PARA_AGENDA]
      ,[U_DIAS_TOPE_PARA_INGRESO_AGENDA]
      ,[M_ATEN_PROV_LUN]
      ,[M_ATEN_PROV_MAR]
      ,[M_ATEN_PROV_MIE]
      ,[M_ATEN_PROV_JUE]
      ,[M_ATEN_PROV_VIE]
      ,[M_ATEN_PROV_SAB]
      ,[M_ATEN_PROV_DOM]
      ,[C_PERIODO_ATENCION_AGENDA]
      ,[U_DIAS_PAGO]
      ,[C_CLIENTE]
      ,[M_COMPENSACION_DIRECTA]
      ,[C_NIVEL_PROVEEDOR]
      ,[C_TIPO_PROVEEDOR_EDI]
      ,[C_TIPO_PROVEEDOR_EDI_CPTE]
  FROM [data-sync].[dbo].[T020_PROVEEDOR];
"""

# Ejecutar la consulta SQL
data = pd.read_sql(query, conn)

# Adecuar Tipos de Datos
data['C_PROVEEDOR']= data['C_PROVEEDOR'].astype(int)
data['C_TIPO_PROVEEDOR']= data['C_TIPO_PROVEEDOR'].astype(int)
data['C_TIPO_PROVEEDOR_DIARCO']= data['C_TIPO_PROVEEDOR_DIARCO'].astype(int)
data['C_LINEA_PRODUCTO']= data['C_LINEA_PRODUCTO'].astype(int)
data['C_PLAZO_ENTREGA1']= data['C_PLAZO_ENTREGA1'].astype(int)
data['C_PLAZO_ENTREGA2']= data['C_PLAZO_ENTREGA2'].astype(int)
data['C_PLAZO_ENTREGA3']= data['C_PLAZO_ENTREGA3'].astype(int)

# Guardar los resultados en un archivo CSV con el nombre de la tienda
file_path = f'data/{label}.csv'
data.to_csv(file_path, index=False, encoding='utf-8')

print(f"Archivo guardado: {file_path}")

# Cerrar la conexión después de la iteración
conn.close()

In [ ]:
# CARGAR DESDE CSV
import chardet

with open('data/T020_Proveedores.csv', 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])  # Muestra la codificación detectada

# Cargar el archivo con codificación UTF-8
proveedores = pd.read_csv('data/T020_Proveedores.csv', encoding='utf-8', sep=',')

# Visualización de las primeras filas\
proveedores.head()

### 3) CARGAR DATOS DE VENTA DEL PROVEEDOR de TODAS LAS SUCURSALES para PRODUCTOS HABILITAOS

#### PROVEEDORES


*   20 - MOLINOS RIO DE LA PLATA
*   25 - CAFES LA VIRGINIA S.A.                            
*   62 - ARCOR
*   98 - FRATELLI BRANCA DESTILERIAS S.A.  
*  140 - UNILEVER DE ARGENTINA S.A.
*  189 - BODEGAS Y VIÑEDOS LOPEZ S.A.I.C.    
* 1465 - QUICKFOOD S.A.             
*  327 - PALADINI S.A.                                     
                                                                                      

In [2]:
proveedores = [
    {"id": 20, "nombre": "MOLINOS RIO DE LA PLATA", "label": "20_MOLINOS"},
    {"id": 25, "nombre": "CAFES LA VIRGINIA S.A.", "label": "25_LA_VIRGINIA"},
    {"id": 62, "nombre": "ARCOR","label":"62_ARCOR", "label": "62_ARCOR"},
    {"id": 98, "nombre": "FRATELLI BRANCA DESTILERIAS S.A.", "label": "98_FRATELLI_BRANCA"},
    {"id": 140, "nombre": "UNILEVER DE ARGENTINA S.A.", "label": "140_UNILEVER"},
    {"id": 189, "nombre": "BODEGAS Y VIÑEDOS LOPEZ S.A.I.C.", "label": "189_BODEGAS_LOPEZ"},
    {"id": 1465, "nombre": "QUICKFOOD S.A.", "label":"1465_QUICKFOOD"},
    {"id": 327, "nombre": "PALADINI S.A.", "label":"327_PALADINI"},
    {"id": 596, "nombre": "PROCTER", "label":"596_PROCTER"},
    {"id": 8449, "nombre": "CAGNOLI S.A.", "label":"8449_CAGNOLI"},
    {"id": 1171, "nombre": "LEDESMA S.A.", "label":"1171_LEDESMA"}
]

for proveedor in proveedores:
    print(f'Procesando: {proveedor["id"]} - {proveedor["nombre"]} - {proveedor["label"]} ')
    # generar_datos(proveedor["id"], proveedor["label"])


Procesando: 20 - MOLINOS RIO DE LA PLATA - 20_MOLINOS 
Procesando: 25 - CAFES LA VIRGINIA S.A. - 25_LA_VIRGINIA 
Procesando: 62 - ARCOR - 62_ARCOR 
Procesando: 98 - FRATELLI BRANCA DESTILERIAS S.A. - 98_FRATELLI_BRANCA 
Procesando: 140 - UNILEVER DE ARGENTINA S.A. - 140_UNILEVER 
Procesando: 189 - BODEGAS Y VIÑEDOS LOPEZ S.A.I.C. - 189_BODEGAS_LOPEZ 
Procesando: 1465 - QUICKFOOD S.A. - 1465_QUICKFOOD 
Procesando: 327 - PALADINI S.A. - 327_PALADINI 
Procesando: 596 - PROCTER - 596_PROCTER 
Procesando: 8449 - CAGNOLI S.A. - 8449_CAGNOLI 
Procesando: 1171 - LEDESMA S.A. - 1171_LEDESMA 


#### ALGORITMOS

In [ ]:
#EJECUCIÓN MASIVA x LISTA
# PONER VENTANA EN DURACIÖN DEL PERÏODO DE TRAIN

algoritmos = [
    {"id": 10, "algoritmo" : "ALGO_01", "ventana": 31, "f1" : 0.80, "f2": 0.10, "f3": 0.10, "Observaciones": "Promedio Ponderado de Ventas de Periodos Anteriores factores (Actual, Anterior y Año Anterior)"},
    {"id": 20, "algoritmo" : "ALGO_02", "ventana": 31, "f1" : 0, "f2": 0, "f3": 0, "Observaciones": "Modelo de Regresión Lineal Doble considera  Tendencia (Holt) Factor ALPHA"},
    {"id": 30, "algoritmo" : "ALGO_03", "ventana": 31, "f1" : 7, "f2": "add", "f3": "add", "Observaciones": "Modelos Regresión Lineal Triple, Considera Estacionalidad y Tendencia (Holt-Winters)"},
    {"id": 40, "algoritmo" : "ALGO_04", "ventana": 31, "f1" : 0.6, "f2": 0, "f3": 0, "Observaciones": "Suavizado Exponencial Simple (SES) + Alpha"},
    {"id": 50, "algoritmo" : "ALGO_05", "ventana": 31, "f1" : 0, "f2": 0, "f3": 0, "Observaciones": "Promedio Venta Simple Ventana Anterior"},
    {"id": 60, "algoritmo" : "ALGO_06", "ventana": 31, "f1" : 0, "f2": 0, "f3": 0, "Observaciones": "Metodo Hols SEMANAL"},
]

for algoritmo in algoritmos:
    print('------------------------------------------------------------------')
    print(f'Procesando: {algoritmo["algoritmo"]}- {algoritmo["Observaciones"]}||--> VENTANA: {algoritmo["ventana"]} - PARAMETROS: {algoritmo["f1"]} - {algoritmo["f2"]} - {algoritmo["f3"]}')

print('------------------------------------------------------------------')


------------------------------------------------------------------
Procesando: ALGO_01- Promedio Ponderado de Ventas de Periodos Anteriores factores (Actual, Anterior y Año Anterior)||--> VENTANA: 31 - PARAMETROS: 0.8 - 0.1 - 0.1
------------------------------------------------------------------
Procesando: ALGO_02- Modelo de Regresión Lineal Doble considera  Tendencia (Holt) Factor ALPHA||--> VENTANA: 31 - PARAMETROS: 0 - 0 - 0.1
------------------------------------------------------------------
Procesando: ALGO_03- Modelos Regresión Lineal Triple, Considera Estacionalidad y Tendencia (Holt-Winters)||--> VENTANA: 31 - PARAMETROS: 7 - add - add
------------------------------------------------------------------
Procesando: ALGO_04- Suavizado Exponencial Simple (SES) + Alpha||--> VENTANA: 31 - PARAMETROS: 0.6 - 0 - 0
------------------------------------------------------------------
Procesando: ALGO_05- Promedio Venta Simple Ventana Anterior||--> VENTANA: 31 - PARAMETROS: 0 - 0 - 0
-----

### 4) GENERACIÓN MASIVA DE DATASETS

In [ ]:
print("🕒 Iniciando ejecución programada del FORECAST ...")
try:
    # Ejecuta la rutina completa
    for proveedor in proveedores:
        print(f'Procesando: {proveedor["id"]} - {proveedor["nombre"]} - {proveedor["label"]} ')
        id_proveedor = proveedor["id"]
        label = proveedor["label"]        
        
        for alg in algoritmos:
            print('      ------------------------------------------------------------------')
            nombre_algoritmo = alg["algoritmo"]
            ventana = alg["ventana"]
            f1 = alg["f1"]
            f2 = alg["f2"]
            f3 = alg["f3"]
            try:
                print(f'      Procesando:{id_proveedor} - {label}  - {nombre_algoritmo} ||--> VENTANA: {ventana}  - PARAMETROS: {f1} - {f2} - {f3}')
                get_forecast(id_proveedor, label, ventana, nombre_algoritmo, f1, f2, f3)
                print("      ✅ Ejecución completada con éxito.")
            except Exception as e:
                print(f"❌ Error durante la ejecución del forecast: {e}")
            
        print('Nuevo Proveedor ------------------------------------------------------------------')
        
    print("✅ ✅ ✅  FIN ✅ ✅ ✅ Ejecución completada con éxito.") 
except Exception as e:
    print(f"❌ Error general al iniciar ejecuciones programadas: {e}")

### 5) DIVISIÓN DATASET en TRAIN Y TEST

In [3]:
# PREPARAR DIVISIÓN TRAINING y TESTING
# CARGAR DESDE CACHE DE DATOS  

print("🕒 Iniciando ejecución programada del FORECAST ...")
try:
    # Ejecuta la rutina completa
    for proveedor in proveedores:
        print(f'Procesando: {proveedor["id"]} -  {proveedor["label"]} ')
        id_proveedor = proveedor["id"]
        label = proveedor["label"]        
        
        try:
            data = pd.read_csv(f'{folder}/{label}.csv')
            data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
            data['Sucursal']= data['Sucursal'].astype(int)
            data['Fecha']= pd.to_datetime(data['Fecha'])

            data_train, data_test = dividir_dataframe(data, '2025-03-01')
            # Generar TRAIN
            file_path = f'{folder}/{label}.csv'
            data_train.to_csv(file_path, index=False, encoding='utf-8')
            print(f"---> Datos TRAINING Guardados: {file_path}")  
            # Generar TEST
            file_path = f'{folder}/{label}_TEST.csv'
            data_test = data_test[['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']]
            data_test.to_csv(file_path, index=False, encoding='utf-8')
            print(f"---> Datos TEST Guardados: {file_path}")  
            
            print("      ✅ División completada con éxito.")
        except Exception as e:
            print(f"❌ Error durante la ejecución del forecast: {e}")
            
        print('Nuevo Proveedor ------------------------------------------------------------------')
        
    print("✅ ✅ ✅  FIN ✅ ✅ ✅ Ejecución completada con éxito.") 
except Exception as e:
    print(f"❌ Error general al iniciar ejecuciones programadas: {e}")

🕒 Iniciando ejecución programada del FORECAST ...
Procesando: 20 -  20_MOLINOS 
---> Datos TRAINING Guardados: data/20_MOLINOS.csv
---> Datos TEST Guardados: data/20_MOLINOS_TEST.csv
      ✅ División completada con éxito.
Nuevo Proveedor ------------------------------------------------------------------
Procesando: 25 -  25_LA_VIRGINIA 
---> Datos TRAINING Guardados: data/25_LA_VIRGINIA.csv
---> Datos TEST Guardados: data/25_LA_VIRGINIA_TEST.csv
      ✅ División completada con éxito.
Nuevo Proveedor ------------------------------------------------------------------
Procesando: 62 -  62_ARCOR 
---> Datos TRAINING Guardados: data/62_ARCOR.csv
---> Datos TEST Guardados: data/62_ARCOR_TEST.csv
      ✅ División completada con éxito.
Nuevo Proveedor ------------------------------------------------------------------
Procesando: 98 -  98_FRATELLI_BRANCA 
---> Datos TRAINING Guardados: data/98_FRATELLI_BRANCA.csv
---> Datos TEST Guardados: data/98_FRATELLI_BRANCA_TEST.csv
      ✅ División compl

### 6) GENERACIÓN MASIVA DE PRONOSTICOS

In [6]:
# POBLAR LOS DICCIONARIOS DE PROVEEDORES Y ALGORITMOS
# DESCONECTAR SQL PARA OBLIGAR A CARGAR DESDE CSV

print("🕒 Iniciando ejecución programada del FORECAST ...")
try:
    # Ejecuta la rutina completa
    for proveedor in proveedores:
        print(f'Procesando: {proveedor["id"]} - {proveedor["nombre"]} - {proveedor["label"]} ')
        id_proveedor = proveedor["id"]
        label = proveedor["label"]        
        
        for alg in algoritmos:
            print('      ------------------------------------------------------------------')
            nombre_algoritmo = alg["algoritmo"]
            ventana = alg["ventana"]
            f1 = alg["f1"]
            f2 = alg["f2"]
            f3 = alg["f3"]
            try:
                print(f'      Procesando:{id_proveedor} - {label}  - {nombre_algoritmo} ||--> VENTANA: {ventana}  - PARAMETROS: {f1} - {f2} - {f3}')
                get_forecast(id_proveedor, label, ventana, nombre_algoritmo, f1, f2, f3)
                print("      ✅ Ejecución completada con éxito.")
            except Exception as e:
                print(f"❌ Error durante la ejecución del forecast: {e}")
            
        print('Nuevo Proveedor ------------------------------------------------------------------')
        
    print("✅ ✅ ✅  FIN ✅ ✅ ✅ Ejecución completada con éxito.") 
except Exception as e:
    print(f"❌ Error general al iniciar ejecuciones programadas: {e}")

🕒 Iniciando ejecución programada del FORECAST ...
Procesando: 20 - MOLINOS RIO DE LA PLATA - 20_MOLINOS 
      ------------------------------------------------------------------
      Procesando:20 - 20_MOLINOS  - ALGO_01 ||--> VENTANA: 31  - PARAMETROS: 0.8 - 0.1 - 0.1
Dentro del get_forecast
FORECAST control: 20 - 20_MOLINOS - ventana: 31 - ALGO_01 factores: 0.8 - 0.1 - 0.1
-> Datos Recuperados del CACHE: 20, Label: 20_MOLINOS
Fecha actual 2025-02-28 00:00:00
--> Procesar_ALGO_01 ventana 31 - Peso de los Factores Utilizados: último: 0.8 previo: 0.1 año anterior: 0.1
Dentro del Calcular_Demanda_ALGO_01
FORECAST control: 20 - 20_MOLINOS - ventana: 31 - factores: 0.8 - 0.1 - 0.1
🖼️ Preparación de Datos - Tiempo: 0.16 seg
🖼️ Demanda Calculada - Tiempo: 0.17 seg
✅ Inserción completada: 3921 registros insertados.
      ✅ Ejecución completada con éxito.
      ------------------------------------------------------------------
      Procesando:20 - 20_MOLINOS  - ALGO_02 ||--> VENTANA: 31  - P

e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


-> ** Datos Exportados: 20_MOLINOS_ALGO_02_Solicitudes_Compra.csv *** : ventana: 31  - 2025-02-28 00:00:00
✅ Inserción completada: 4067 registros insertados.
      ✅ Ejecución completada con éxito.
      ------------------------------------------------------------------
      Procesando:20 - 20_MOLINOS  - ALGO_03 ||--> VENTANA: 31  - PARAMETROS: 7 - add - add
Dentro del get_forecast
FORECAST control: 20 - 20_MOLINOS - ventana: 31 - ALGO_03 factores: 7 - add - add
-> Datos Recuperados del CACHE: 20, Label: 20_MOLINOS
Fecha actual 2025-02-28 00:00:00
--> Procesar_ALGO_03 ventana 31 - Factores Utilizados: Períodos: 7 estacionalidad: add tendencia: add
Dentro del Calcular_Demanda_ALGO_03
FORECAST control: 20 - 20_MOLINOS - ventana: 31 - factores: Períodos Estacionalidad  7 - Tendencia: add - Estacionalidad: add
-> ** Datos Exportados: 20_MOLINOS_ALGO_03_Solicitudes_Compra.csv *** : ventana: 31  - 2025-02-28 00:00:00
✅ Inserción completada: 4067 registros insertados.
      ✅ Ejecución compl

e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
e:\PY\.venv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


-> ** Solicitudes Exportadas: 20_MOLINOS_ALGO_06_Solicitudes_Compra.csv *** : ventana: 31  - 2025-02-28 00:00:00
✅ Inserción completada: 4061 registros insertados.
      ✅ Ejecución completada con éxito.
Nuevo Proveedor ------------------------------------------------------------------
Procesando: 25 - CAFES LA VIRGINIA S.A. - 25_LA_VIRGINIA 
      ------------------------------------------------------------------
      Procesando:25 - 25_LA_VIRGINIA  - ALGO_01 ||--> VENTANA: 31  - PARAMETROS: 0.8 - 0.1 - 0.1
Dentro del get_forecast
FORECAST control: 25 - 25_LA_VIRGINIA - ventana: 31 - ALGO_01 factores: 0.8 - 0.1 - 0.1
-> Datos Recuperados del CACHE: 25, Label: 25_LA_VIRGINIA
Fecha actual 2025-02-28 00:00:00
--> Procesar_ALGO_01 ventana 31 - Peso de los Factores Utilizados: último: 0.8 previo: 0.1 año anterior: 0.1
Dentro del Calcular_Demanda_ALGO_01
FORECAST control: 25 - 25_LA_VIRGINIA - ventana: 31 - factores: 0.8 - 0.1 - 0.1
🖼️ Preparación de Datos - Tiempo: 0.02 seg
🖼️ Demanda Cal

### 7) COMPAGINACIÓN DATOS PARA ANÁLIZAR EFECTIVIDAD

In [2]:
# FUNCIONES DE PROCESAMIENTO DE RESULTADOS
import os
import time
import pandas as pd
import numpy as np
from dotenv import dotenv_values
import pyodbc
import psycopg2 as pg2


# Función de conexión a SQL Server
def Open_Connection():
    secrets = dotenv_values(".env")
    conn_str = f'DRIVER={secrets["DRIVER2"]};SERVER={secrets["SERVIDOR2"]};PORT={secrets["PUERTO2"]};DATABASE={secrets["BASE2"]};UID={secrets["USUARIO2"]};PWD={secrets["CONTRASENA2"]}'
    try:
        conn = pyodbc.connect(conn_str)
        return conn
    except Exception as e:
        print(f"Error en la conexión SQL Server: {e}")
        return None

# Función de conexión a PostgreSQL
def Open_Conn_Postgres():
    secrets = dotenv_values(".env")
    conn_str = f"dbname={secrets['BASE4']} user={secrets['USUARIO4']} password={secrets['CONTRASENA4']} host={secrets['SERVIDOR4']} port={secrets['PUERTO4']}"
    for i in range(5):
        try:
            conn = pg2.connect(conn_str)
            return conn
        except Exception as e:
            print(f"Error en la conexión PostgreSQL, intento {i+1}/5: {e}")
            time.sleep(5)
    return None

# Carga de todos los archivos CSV de forecast
def cargar_todos_los_forecasts(carpeta_path, id_proveedor):
    dfs = []
    for file in os.listdir(carpeta_path):
        if file.startswith(str(id_proveedor)) and file.endswith("Compra.csv"):
            df = pd.read_csv(os.path.join(carpeta_path, file))
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Extracción de ventas desde SQL Server
def extraer_ventas(id_proveedor, test_start, test_end):
    query = f"""
    SELECT MAX(V.[F_VENTA]) AS Fecha,
        V.[C_ARTICULO] AS Codigo_Articulo,
        V.[C_SUCU_EMPR] AS Sucursal,
        SUM(V.[Q_UNIDADES_VENDIDAS]) AS Unidades
    FROM [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
    LEFT JOIN [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] A 
        ON V.[C_ARTICULO] = A.[C_ARTICULO]
    WHERE A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} 
        AND V.[F_VENTA] >= '{test_start}' 
        AND V.[F_VENTA] <= '{test_end}'
        AND A.[M_BAJA] = 'N'
    GROUP BY V.[F_VENTA], V.[C_ARTICULO], V.[C_SUCU_EMPR]
    
    """
    conn = Open_Connection()
    if conn:
        df_ventas = pd.read_sql(query, conn)
        conn.close()
        df_ventas['Fecha'] = pd.to_datetime(df_ventas['Fecha'])
        df_ventas['Semana'] = df_ventas['Fecha'].dt.isocalendar().week
        return df_ventas
    else:
        return pd.DataFrame()

# Cálculo de métricas por SKU y algoritmo
def consolidar_metricas_para_todos(df_forecast, df_ventas):
    #df_forecast['Semana'] = df_forecast['ventana']
    df = pd.merge(
        df_forecast,
        df_ventas,
        how='inner',
        on=['Codigo_Articulo', 'Sucursal']
    )
    df['Error_Absoluto'] = np.abs(df['Forecast'] - df['Unidades'])
    df['Error_Cuadratico'] = (df['Forecast'] - df['Unidades']) ** 2
    df['sMAPE'] = 2 * np.abs(df['Forecast'] - df['Unidades']) / (
        np.abs(df['Forecast']) + np.abs(df['Unidades']) + 1e-6
    )

    resumen = df.groupby(
        ['Codigo_Articulo', 'algoritmo'], as_index=False
    ).agg({
        'Error_Absoluto': 'mean',
        'Error_Cuadratico': 'mean',
        'sMAPE': 'mean'
    })

    resumen.rename(columns={
        'Error_Absoluto': 'MAE',
        'Error_Cuadratico': 'RMSE'
    }, inplace=True)
    resumen['RMSE'] = np.sqrt(resumen['RMSE'])

    return resumen

# Selección del mejor algoritmo por SKU
def seleccionar_mejor_algoritmo(df_metricas, criterio='MAE'):
    df_metricas['Rank'] = df_metricas.groupby('Codigo_Articulo')[criterio].rank(method='min')
    df_mejor = df_metricas[df_metricas['Rank'] == 1].drop(columns='Rank')
    df_mejor.rename(columns={'algoritmo': 'Algoritmo_Optimo'}, inplace=True)
    df_mejor['Fecha_Evaluacion'] = pd.Timestamp.today().normalize()
    return df_mejor

# Guardar resultado en PostgreSQL
def guardar_en_postgres(df, tabla_destino):
    conn = Open_Diarco_Data()
    if conn is not None:
        cursor = conn.cursor()
        for _, row in df.iterrows():
            cursor.execute(f"""
                INSERT INTO {tabla_destino} 
                (Codigo_Articulo, Algoritmo_Optimo, MAE, RMSE, sMAPE, Fecha_Evaluacion)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Codigo_Articulo) DO UPDATE 
                SET Algoritmo_Optimo = EXCLUDED.Algoritmo_Optimo,
                    MAE = EXCLUDED.MAE,
                    RMSE = EXCLUDED.RMSE,
                    sMAPE = EXCLUDED.sMAPE,
                    Fecha_Evaluacion = EXCLUDED.Fecha_Evaluacion;
            """, (
                int(row['Codigo_Articulo']),
                row['Algoritmo_Optimo'],
                float(row['MAE']),
                float(row['RMSE']),
                float(row['sMAPE']),
                row['Fecha_Evaluacion']
            ))
        conn.commit()
        cursor.close()
        conn.close()
        print("Datos cargados en PostgreSQL.")
    else:
        print("No se pudo establecer conexión con PostgreSQL.")
        
# === ANÁLISIS GLOBAL POR PROVEEDOR ===
def evaluar_algoritmo_globa2l(df_metricas):
    resumen_global = df_metricas.groupby('algoritmo').agg({
        'MAE': 'mean',
        'RMSE': 'mean',
        'sMAPE': 'mean'
    }).reset_index()

    print("\nResumen global de métricas por algoritmo:")
    print(resumen_global.sort_values('MAE'))
    
    mejor = resumen_global.sort_values('MAE').iloc[0]
    print(f"\n>>> El mejor algoritmo global para el proveedor es: {mejor['algoritmo']} (MAE promedio = {mejor['MAE']:.2f})")
    return resumen_global

def evaluar_algoritmo_global(df_metricas, df_mejor_algoritmo):
    resumen_global = df_metricas.groupby('algoritmo').agg({
        'MAE': 'mean',
        'RMSE': 'mean',
        'sMAPE': 'mean'
    }).reset_index()

    # Agregar el conteo de veces que cada algoritmo fue el óptimo
    conteo = df_mejor_algoritmo['Algoritmo_Optimo'].value_counts().reset_index()
    conteo.columns = ['algoritmo', 'Veces_mejor']

    # Unir el conteo con el resumen global
    resumen_global = resumen_global.merge(conteo, on='algoritmo', how='left')
    resumen_global['Veces_mejor'] = resumen_global['Veces_mejor'].fillna(0)  # Rellenar NaN con 0
    print ('---------------------------------------------------------------------')
    print("\nResumen global de métricas por algoritmo:")
    print(resumen_global.sort_values('MAE'))
    print ('---------------------------------------------------------------------')
    mejor = resumen_global.sort_values('MAE').iloc[0]
    print(f"\n>>> El mejor algoritmo global para el proveedor es: {mejor['algoritmo']} (MAE promedio = {mejor['MAE']:.2f})")

    return resumen_global




### Explicación
Retorno: La función devuelve un dataframe , con las métricas calculadas por cada artículo y algoritmo:

* MAE (Mean Absolute Error): Promedio del error absoluto.
* RMSE (Root Mean Squared Error): Promedio del error cuadrático en escala original.
* sMAPE: Promedio de error porcentual simétrico.


Los valores óptimos de estas métricas dependen del contexto y del tipo de datos que se estén analizando, pero aquí te doy una guía general sobre cómo interpretarlos:

- **MAE (Mean Absolute Error)** y **RMSE (Root Mean Squared Error)**:  
  - Valores bajos indican que el modelo tiene una buena precisión.  
  - No hay un umbral absoluto para un "buen" o "mal" resultado, pero generalmente, si el error es pequeño en comparación con la escala de los datos, el pronóstico es considerado aceptable.  
  - **RMSE suele ser mayor que el MAE**, ya que penaliza más los errores grandes.  

- **sMAPE (Symmetric Mean Absolute Percentage Error)**:  
  - Se expresa en porcentaje, por lo que es más fácil de interpretar.  
  - Un valor bajo (por ejemplo, **menos del 20%**) suele indicar una buena precisión.  
  - Un valor por encima del 50% generalmente sugiere que el pronóstico no es confiable.  

En general, lo ideal es comparar estos valores con modelos previos o benchmarks de la industria para saber si los errores están dentro de un rango aceptable. ¿Estás evaluando un pronóstico específico? Puedo ayudarte a analizarlo en más detalle.

 🚀 ✨ ✅  🔍 

In [15]:
# Main
if __name__ == "__main__":
    id_proveedor = 8449
    carpeta_forecasts = folder
    tabla_destino = "forecast_algoritmo_optimo"
    ventana = 31  # Ventana de entrenamiento (en días)
    print("Cargando predicciones...")
    df_forecast = cargar_todos_los_forecasts(carpeta_forecasts, id_proveedor)
    
    # Definir rangos de fechas para cada período
    fecha = df_forecast['Fecha_Pronostico'].max()  # Se toma la última fecha en los datos
    fecha = pd.to_datetime(fecha)

    test_start = fecha + pd.Timedelta(days=ventana - 1)
    test_end = fecha + pd.Timedelta(days=ventana)
    print(f"Rango de fechas para el período de prueba: {test_start} - {test_end}")
    
    print("Extrayendo ventas reales...")
    df_ventas = extraer_ventas(id_proveedor, test_start, test_end)

    print("Calculando métricas...")
    df_metricas = consolidar_metricas_para_todos(df_forecast, df_ventas)

    print("Seleccionando el mejor algoritmo por artículo...")
    df_mejor_algoritmo = seleccionar_mejor_algoritmo(df_metricas, criterio='MAE')

    print("Guardando resultados...")
    guardar_en_postgres(df_mejor_algoritmo, tabla_destino)
    
    print("Analizando mejor algoritmo a nivel proveedor...", id_proveedor)
    evaluar_algoritmo_global(df_metricas, df_mejor_algoritmo)
    print ('---------------------------------------------------------------------')

    print("Proceso finalizado.")
    

Cargando predicciones...
Rango de fechas para el período de prueba: 2025-03-30 00:00:00 - 2025-03-31 00:00:00
Extrayendo ventas reales...
Calculando métricas...
Seleccionando el mejor algoritmo por artículo...
Guardando resultados...
Datos cargados en PostgreSQL.
Analizando mejor algoritmo a nivel proveedor... 8449
---------------------------------------------------------------------

Resumen global de métricas por algoritmo:
  algoritmo        MAE        RMSE     sMAPE  Veces_mejor
0   ALGO_01  19.381370   21.983569  1.534569         15.0
5   ALGO_06  20.959033   26.020535  1.483310         18.0
4   ALGO_05  23.405589   25.964925  1.625387          2.0
2   ALGO_03  26.830017   30.769518  1.625027          2.0
3   ALGO_04  57.847612   67.551452  1.827323          0.0
1   ALGO_02  88.806901  115.788833  1.847290          2.0
---------------------------------------------------------------------

>>> El mejor algoritmo global para el proveedor es: ALGO_01 (MAE promedio = 19.38)
----------

### ANEXO

In [9]:
import pandas as pd
from datetime import datetime, timedelta

# Suponiendo que ya tienes tu DataFrame cargado como 'data'

# Determinar el último día del mes actual
ultimo_dia_mes = data['Fecha'].max()

# Determinar los límites de las fechas
primer_dia_mes_actual = ultimo_dia_mes.replace(day=1)
primer_dia_mes_anterior = (primer_dia_mes_actual - timedelta(days=1)).replace(day=1)

# Crear particiones
ultimo_mes = data[(data['Fecha'] >= primer_dia_mes_actual) & (data['Fecha'] <= ultimo_dia_mes)]
mes_anterior = data[(data['Fecha'] >= primer_dia_mes_anterior) & (data['Fecha'] < primer_dia_mes_actual)]
resto = data[data['Fecha'] < primer_dia_mes_anterior]

# Opcional: Verificar tamaño de cada partición
print(f"Último Mes: {len(ultimo_mes)} registros")
print(f"Mes Anterior: {len(mes_anterior)} registros")
print(f"Resto: {len(resto)} registros")


Último Mes: 1278 registros
Mes Anterior: 9281 registros
Resto: 220965 registros


In [10]:
print (f"Ultimo Mes: {ultimo_mes['Fecha'].min()} - {ultimo_mes['Fecha'].max()}")
print (f"Mes Anterior: {mes_anterior['Fecha'].min()} - {mes_anterior['Fecha'].max()}")  

Ultimo Mes: 2025-04-01 00:00:00 - 2025-04-04 00:00:00
Mes Anterior: 2025-03-01 00:00:00 - 2025-03-31 00:00:00


In [ ]:
# Determinar el último día del mes actual
ultimo_dia_mes = data['Fecha'].max()

print(f"Ultimo Dia del Mes: {ultimo_dia_mes}")


# Ejemplo de uso
fecha_corte = '2025-03-01'
data_train, data_test = dividir_dataframe(data, fecha_corte)

# Ver tamaño de las particiones
print(f"Data Train: {len(data_train)} registros")
print(f"Data Test: {len(data_test)} registros")

Ultimo Dia del Mes: 2025-04-04 00:00:00


In [ ]:
# Configuración de conexión
conn = Open_Connection()

# FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
query = f"""
SELECT  A.[C_PROVEEDOR_PRIMARIO]
	 ,S.[C_ARTICULO]
      ,S.[C_SUCU_EMPR]
      ,S.[Q_FACTOR_VENTA_ESP]
      ,S.[Q_FACTOR_VTA_SUCU]
      ,S.[M_OFERTA_SUCU]
      ,S.[M_HABILITADO_SUCU]
	 ,A.M_BAJA
      ,S.[Q_VTA_DIA_ANT]
      ,S.[Q_VTA_ACUM]
      ,S.[Q_ULT_ING_STOCK]
      ,S.[Q_STOCK_A_ULT_ING]
      ,S.[Q_15DIASVTA_A_ULT_ING_STOCK]
      ,S.[Q_30DIASVTA_A_ULT_ING_STOCK]
      ,S.[Q_BULTOS_PENDIENTE_OC]
      ,S.[Q_PESO_PENDIENTE_OC]
      ,S.[Q_UNID_PESO_PEND_RECEP_TRANSF]
      ,S.[Q_UNID_PESO_VTA_MES_ACTUAL]
      ,S.[F_ULTIMA_VTA]
      ,S.[Q_VTA_ULTIMOS_15DIAS]
      ,S.[Q_VTA_ULTIMOS_30DIAS]
      ,S.[Q_TRANSF_PEND]
      ,S.[Q_TRANSF_EN_PREP]
      ,S.[M_FOLDER]
      ,S.[M_ALTA_RENTABILIDAD]
      ,S.[Lugar_Abastecimiento]
      ,S.[M_COSTO_LOGISTICO]
 
 FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
 LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
	ON A.[C_ARTICULO] = S.[C_ARTICULO]

 WHERE S.[M_HABILITADO_SUCU] = 'S' -- Permitido Reponer
	AND A.M_BAJA = 'N'  -- Activo en Maestro Artículos
	AND A.[C_PROVEEDOR_PRIMARIO] = {proveedor} -- Solo del Proveedor
;
"""

# Ejecutar la consulta SQL
articulos = pd.read_sql(query, conn)



In [ ]:
# Consulta SQL para obtener las ventas de un proveedor específico   
# Reemplazar {proveedor} en la consulta con el ID de la tienda actual
query = f"""
SELECT V.[F_VENTA] as Fecha
     ,V.[C_ARTICULO] as Codigo_Articulo
     ,V.[C_SUCU_EMPR] as Sucursal
     ,V.[I_PRECIO_VENTA] as Precio
     ,V.[I_PRECIO_COSTO] as Costo
     ,V.[Q_UNIDADES_VENDIDAS] as Unidades
     ,V.[C_FAMILIA] as Familia
     ,A.[C_RUBRO] as Rubro
     ,A.[C_SUBRUBRO_1] as SubRubro
     ,LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(A.N_ARTICULO, CHAR(9), ''), CHAR(13), ''), CHAR(10), ''))) as Nombre_Articulo
     ,A.[C_CLASIFICACION_COMPRA] as Clasificacion
FROM [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
LEFT JOIN [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] A 
     ON V.C_ARTICULO = A.C_ARTICULO
WHERE A.[C_PROVEEDOR_PRIMARIO] = {proveedor} AND V.F_VENTA >= '20210101' AND A.M_BAJA ='N'
ORDER BY V.F_VENTA ;
"""

# Ejecutar la consulta SQL
demanda = pd.read_sql(query, conn)

In [ ]:
articulos.info()

demanda.info()

In [ ]:
# UNIR Y FILTRAR solo la demanda de los Hartículos VALIDOS.
# Realizar la unión (merge) de los DataFrames por las claves especificadas
data = pd.merge(
    articulos,  # DataFrame de artículos
    demanda,    # DataFrame de demanda
    left_on=['C_ARTICULO', 'C_SUCU_EMPR'],  # Claves en 'articulos'
    right_on=['Codigo_Articulo', 'Sucursal'],  # Claves en 'demanda'
    how='inner'  # Solo traer los productos que están en 'articulos'
)
# Mostrar el DataFrame resultante
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Data - Artículos con Demanda", dataframe=data)


In [ ]:
# HISTORIA de DEMANDA FILTRADA por ARTICULOS NO HABILITADOS PARA COMPRAR
# Cantidad de artículos únicos antes del merge
articulos_unicos = demanda[['Codigo_Articulo', 'Sucursal']].drop_duplicates().shape[0]

# Cantidad de artículos únicos después del merge
articulos_unidos = data[['C_ARTICULO', 'C_SUCU_EMPR']].drop_duplicates().shape[0]

# Calcular cuántos artículos fueron excluidos
articulos_excluidos = articulos_unicos - articulos_unidos

print(f"Artículos en 'demanda' antes del merge: {articulos_unicos}")
print(f"Artículos en 'data' después del merge: {articulos_unidos}")
print(f"Artículos excluidos: {articulos_excluidos}")

In [ ]:
# Guardar los resultados en un archivo CSV con el nombre de la tienda
file_path = f'data/{label}.csv'
data.to_csv(file_path, index=False, encoding='utf-8')

print(f"Archivo guardado: {file_path}")

# Cerrar la conexión después de la iteración
conn.close()

## División de Datos en Entrenamiento y Prueba

Se define un corte temporal para separar el conjunto en datos de entrenamiento y de prueba. En este ejemplo, se utiliza el último mes de datos como conjunto de prueba.

In [ ]:
train_end = df_model.index.max() - pd.Timedelta(days=30)
train_data = df_model.loc[:train_end]
test_data = df_model.loc[train_end + pd.Timedelta(days=1):]

print('Periodo de Entrenamiento:', train_data.index.min(), 'a', train_data.index.max())
print('Periodo de Prueba:', test_data.index.min(), 'a', test_data.index.max())